In [3]:
import pandas as pd
import numpy as np
import os

# Change OS directory to match wherever you downloaded the files

os.chdir("C:\\Users\\Anna\\Downloads")

df_1 = pd.read_csv("movies_data\\tmdb_5000_movies.csv")
df_2 = pd.read_csv("movies_data_2\\TMDB_movie_dataset_v11.csv")

#print("Movie Dataset 1:\n")
#print(df_1.head())
#print("Movie Dataset 2:\n")
#print(df_2.head())

# Clean each dataset to only include relevant info

df_1 = df_1[['title', 'budget', 'original_language', 'release_date', 'revenue', 'runtime']]
#print("\nMovie Dataset 1:\n")
#print(df_1.head())
df_2 = df_2[['title', 'genres', 'adult']]
#print("\nMovie Dataset 2:\n")
#print(df_2.head())

# Merge datasets and clean

movies_df = pd.merge(df_1, df_2, on='title', how='left')
movies_df = movies_df.drop_duplicates(subset=['title'])
movies_df['is_adult?'] = np.where(
    movies_df['adult'].astype(str).str.lower() == 'True',
    1,
    0
)
movies_df.drop(columns=['adult'], inplace=True)
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')
movies_df['release_date'] = movies_df['release_date'].dt.month_name()
print("\nMovies Dataframe:\n")
#print(movies_df)

# Create new "success/failure" column

movies_df['is_success?'] = (movies_df['revenue'] >= (movies_df['budget'] * 2)).astype(int)
print(movies_df)





Movies Dataframe:

                                          title     budget original_language  \
0                                        Avatar  237000000                en   
6      Pirates of the Caribbean: At World's End  300000000                en   
7                                       Spectre  245000000                en   
17                        The Dark Knight Rises  250000000                en   
18                                  John Carter  260000000                en   
...                                         ...        ...               ...   
17649                               El Mariachi     220000                es   
17650                                 Newlyweds       9000                en   
17652                 Signed, Sealed, Delivered          0                en   
17653                          Shanghai Calling          0                en   
17654                         My Date with Drew          0                en   

      release_date 

In [7]:
# Split all columns into 0/1 categories


from sklearn.preprocessing import MultiLabelBinarizer

# Created a function to make all the genres atttrubutes a 'list' datatype in order for MLB^^^ to work

def clean_genres(x):
    if isinstance(x, str):
        return [g.strip() for g in x.split(',') if g.strip()]
    else:
        return []

movies_df['genres'] = movies_df['genres'].apply(clean_genres)

movies_df['genres'].apply(type).value_counts()


genres
<class 'list'>    4800
Name: count, dtype: int64

In [13]:
# Splitting columns (continued)

#

mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(movies_df['genres'])

genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)
movies_df = movies_df.join(genres_df)
movies_df = movies_df.drop(columns=['genres'])
movies_df.head()

KeyError: 'genres'